## GAN Example to generate MNIST data
- Ref - https://github.com/malhamid/GAN_MNIST/blob/master/GAN_MNIST.ipynb

In [3]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout
from tensorflow.keras.layers import Conv2DTranspose, Reshape, LeakyReLU

### Constant

In [27]:
INPUT_SHAPE = (28, 28, 1)
NOISE_DIM = (100, )

### Load Dataset

In [4]:
# Train -- (60000, 28, 28), (60000, )
# Test -- (10000, 28, 28), (10000, )
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


### Data Preprocessing

In [9]:
# (N, 28, 28) => (N, 28*28)
X_train = np.reshape(X_train, (X_train.shape[0], 28 * 28)) / 255.0
X_test = np.reshape(X_test, (X_test.shape[0], 28 * 28)) / 255.0

### Create Model

In [29]:
# Discriminator
def building_discriminator():
    model_d = Sequential()
    # Layer 1
    model_d.add(Conv2D(64, 3, 2, input_shape=INPUT_SHAPE))
    model_d.add(LeakyReLU())
    model_d.add(Dropout(0.4))

    # Layer 2
    model_d.add(Conv2D(64, 3, 2))
    model_d.add(LeakyReLU())
    model_d.add(Dropout(0.4))

    # Final Classification Layer
    model_d.add(Flatten())
    model_d.add(Dense(1, activation='sigmoid'))

    # Compile
    model_d.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model_d

In [25]:
# Generator
def building_generator():
    model_g = Sequential()
    # Layer 1
    model_g.add(Dense(128*6*6, input_shape=NOISE_DIM))
    model_g.add(LeakyReLU())
    model_g.add(Reshape((6, 6, 128)))

    # Layer 2
    model_g.add(Conv2DTranspose(128, (4, 4), (2, 2)))
    model_g.add(LeakyReLU())

    # Layer 3
    model_g.add(Conv2DTranspose(128, (4, 4), (2, 2)))
    model_g.add(LeakyReLU())
    model_g.add(Conv2D(1, (3, 3), activation='sigmoid'))

    return model_g    

In [23]:
def building_gan(generator, discriminator):
    gan = Sequential()
    discriminator.trainable = False

    gan.add(generator)
    gan.add(discriminator)
    gan.compile(loss='binary_crossentropy', optimizer='adam')

    return gan

In [30]:
# Create GAN Model
gen = building_generator()
dis = building_discriminator()
gan = building_gan(gen, dis)
gan.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_3 (Sequential)    (None, 28, 28, 1)         991105    
_________________________________________________________________
sequential_4 (Sequential)    (None, 1)                 39873     
Total params: 1,030,978
Trainable params: 991,105
Non-trainable params: 39,873
_________________________________________________________________


### Train Model